# Transfer Learning

In this notebook, you'll learn how to use pre-trained networks to solved challenging problems in computer vision. Specifically, you'll use networks trained on [ImageNet](http://www.image-net.org/) [available from torchvision](http://pytorch.org/docs/0.3.0/torchvision/models.html). 

ImageNet is a massive dataset with over 1 million labeled images in 1000 categories. It's used to train deep neural networks using an architecture called convolutional layers. I'm not going to get into the details of convolutional networks here, but if you want to learn more about them, please [watch this](https://www.youtube.com/watch?v=2-Ol7ZB0MmU).

Once trained, these models work astonishingly well as feature detectors for images they weren't trained on. Using a pre-trained network on images not in the training set is called transfer learning. Here we'll use transfer learning to train a network that can classify our cat and dog photos with near perfect accuracy.

With `torchvision.models` you can download these pre-trained networks and use them in your applications. We'll include `models` in our imports now.

In [53]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

Most of the pretrained models require the input to be 224x224 images. Also, we'll need to match the normalization used when the models were trained. Each color channel was normalized separately, the means are `[0.485, 0.456, 0.406]` and the standard deviations are `[0.229, 0.224, 0.225]`.

In [54]:
data_dir = 'Cat_Dog_data'

# TODO: Define transforms for the training data and testing data

train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])])


# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=32)

We can load in a model such as [DenseNet](http://pytorch.org/docs/0.3.0/torchvision/models.html#id5). Let's print out the model architecture so we can see what's going on.

In [3]:
model = models.densenet121(pretrained=True)
model

/opt/conda/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.torch/models/densenet121-a639ec97.pth
100%|██████████| 32342954/32342954 [00:01<00:00, 17285140.00it/s]


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

This model is built out of two main parts, the features and the classifier. The features part is a stack of convolutional layers and overall works as a feature detector that can be fed into a classifier. The classifier part is a single fully-connected layer `(classifier): Linear(in_features=1024, out_features=1000)`. This layer was trained on the ImageNet dataset, so it won't work for our specific problem. That means we need to replace the classifier, but the features will work perfectly on their own. In general, I think about pre-trained networks as amazingly good feature detectors that can be used as the input for simple feed-forward classifiers.

In [4]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

With our model built, we need to train the classifier. However, now we're using a **really deep** neural network. If you try to train this on a CPU like normal, it will take a long, long time. Instead, we're going to use the GPU to do the calculations. The linear algebra computations are done in parallel on the GPU leading to 100x increased training speeds. It's also possible to train on multiple GPUs, further decreasing training time.

PyTorch, along with pretty much every other deep learning framework, uses [CUDA](https://developer.nvidia.com/cuda-zone) to efficiently compute the forward and backwards passes on the GPU. In PyTorch, you move your model parameters and other tensors to the GPU memory using `model.to('cuda')`. You can move them back from the GPU with `model.to('cpu')` which you'll commonly do when you need to operate on the network output outside of PyTorch. As a demonstration of the increased speed, I'll compare how long it takes to perform a forward and backward pass with and without a GPU.

In [5]:
import time

In [6]:
for device in ['cpu', 'cuda']:

    criterion = nn.NLLLoss()
    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

    model.to(device)

    for ii, (inputs, labels) in enumerate(trainloader):

        # Move input and label tensors to the GPU
        inputs, labels = inputs.to(device), labels.to(device)

        start = time.time()

        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if ii==3:
            break
        
    print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")

Device = cpu; Time per batch: 5.430 seconds
Device = cuda; Time per batch: 0.010 seconds


You can write device agnostic code which will automatically use CUDA if it's enabled like so:
```python
# at beginning of the script
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

...

# then whenever you get a new Tensor or Module
# this won't copy if they are already on the desired device
input = data.to(device)
model = MyModule(...).to(device)
```

From here, I'll let you finish training the model. The process is the same as before except now your model is much more powerful. You should get better than 95% accuracy easily.

>**Exercise:** Train a pretrained models to classify the cat and dog images. Continue with the DenseNet model, or try ResNet, it's also a good model to try out first. Make sure you are only training the classifier and the parameters for the features part are frozen.

In [8]:
model = models.densenet121(pretrained=True)
#model = models.resnet18(pretrained=True)
print(model)

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu1', nn.ReLU()),
                          ('fc2', nn.Linear(500, 250)),
                          ('relu2', nn.ReLU()),
                          ('fc3', nn.Linear(250, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier
#print(model)

/opt/conda/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [9]:
# TODO: Train a model with a pre-trained network
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.classifier.parameters(), lr=1e-3) #, betas=(0.9, 0.999), eps=1e-8,
                 #weight_decay=0, amsgrad=False)   #only optimizing classifiers paramters from main convlution not changed and frozen

#model.to(device)

model.to('cuda')

epochs = 3


print_every = 40
for e in range(epochs):
    running_loss = 0
    #.train()
    accuracy=0.0
    steps = 0
    for ii, (inputs, labels) in enumerate(trainloader): 
        if (ii>=0):
            steps += 1

            
            inputs, labels = inputs.to('cuda'), labels.to('cuda')

            optimizer.zero_grad()

            output = model.forward(inputs)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            ps = torch.exp(output)
            # Class with highest probability is our predicted class, compare with true label
            equality = (labels.data == ps.max(1)[1])
            # Accuracy is number of correct predictions divided by all predictions, just take the mean (no is not, you need to divide by sample size here to be called accuracy)
            accuracy += equality.type_as(torch.FloatTensor()).mean()
            if ii % print_every ==0:
                print('accuracy %f ,pic %d ,epoch %d /%d' %(accuracy.item()/(ii+1),ii,e+1,epochs))
            #if steps % print_every == 0:
            #    print(accuracy/ii)
            #if steps % print_every == 0:
                # Make sure network is in eval mode for inference
                #model.eval()

                # Turn off gradients for validation, saves memory and computations
                #with torch.no_grad():
                #    test_loss, accuracy = validation(model, testloader, criterion)

                #print("Epoch: {}/{}.. ".format(e+1, epochs),
                #      "Training Loss: {:.3f}.. ".format(running_loss/print_every),
                #      "Train Accuracy: {:.3f}".format(accuracy))/ii    #len(trainloader)  #taking it to validation function

      # "Test Loss: {:.3f}.. ".format(test_loss),    #/len(testloader)),

accuracy 0.734375 ,pic 0 ,epoch 1 /3
accuracy 0.866235 ,pic 40 ,epoch 1 /3
accuracy 0.888310 ,pic 80 ,epoch 1 /3
accuracy 0.897340 ,pic 120 ,epoch 1 /3
accuracy 0.902756 ,pic 160 ,epoch 1 /3
accuracy 0.908582 ,pic 200 ,epoch 1 /3
accuracy 0.910659 ,pic 240 ,epoch 1 /3
accuracy 0.911088 ,pic 280 ,epoch 1 /3
accuracy 0.913600 ,pic 320 ,epoch 1 /3
accuracy 0.906250 ,pic 0 ,epoch 2 /3
accuracy 0.938262 ,pic 40 ,epoch 2 /3
accuracy 0.941165 ,pic 80 ,epoch 2 /3
accuracy 0.936338 ,pic 120 ,epoch 2 /3
accuracy 0.933036 ,pic 160 ,epoch 2 /3
accuracy 0.933302 ,pic 200 ,epoch 2 /3
accuracy 0.933415 ,pic 240 ,epoch 2 /3
accuracy 0.934164 ,pic 280 ,epoch 2 /3
accuracy 0.934774 ,pic 320 ,epoch 2 /3
accuracy 0.953125 ,pic 0 ,epoch 3 /3
accuracy 0.943216 ,pic 40 ,epoch 3 /3
accuracy 0.941551 ,pic 80 ,epoch 3 /3
accuracy 0.940083 ,pic 120 ,epoch 3 /3
accuracy 0.939538 ,pic 160 ,epoch 3 /3
accuracy 0.941076 ,pic 200 ,epoch 3 /3
accuracy 0.939834 ,pic 240 ,epoch 3 /3
accuracy 0.940169 ,pic 280 ,epoch 3 /

In [50]:
model.to('cuda')
correct = 0
total = 0
accuracy=0
with torch.no_grad():
    #for inputs,labels in testloader:
    for ii, (inputs, labels) in enumerate(testloader):
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        #images, labels = data
        outputs = model.forward(inputs)
        #ps = torch.exp(outputs)
        ps = torch.max(outputs.data, 1)[1]
        if(ii>0):
            #print(labels)
            #print(outputs)
            #print(ps)
            #print('')
            #print(ps.max(1)[1])
            equality = (labels == ps)
                    
            accuracy += equality.type_as(torch.FloatTensor()).mean().item()  #  .mean()
            #_, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            #correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %f ' % (100 * accuracy / total))



Accuracy of the network on the 10000 test images: 1.611882 


In [ ]:
            
            # Class with highest probability is our predicted class, compare with true label
            equality = (labels.data == ps.max(1)[1])
            # Accuracy is number of correct predictions divided by all predictions, just take the mean (no is not, you need to divide by sample size here to be called accuracy)
            accuracy += equality.type_as(torch.FloatTensor()).mean()

In [3]:
model = models.densenet121(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.classifier.parameters(), lr=1e-3) #, betas=(0.9, 0.999), eps=1e-8,
 

/opt/conda/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.torch/models/densenet121-a639ec97.pth
100%|██████████| 32342954/32342954 [00:01<00:00, 21685115.04it/s]


In [56]:
# Putting the above into functions, so they can be used later

def do_deep_learning(model, trainloader, epochs, print_every, criterion, optimizer, device='cpu'):
    epochs = epochs
    print_every = print_every
    steps = 0
    model=model

    # change to cuda
    model.to('cuda')

    for e in range(epochs):
        running_loss = 0
        for ii, (inputs, labels) in enumerate(trainloader):
            steps += 1

            inputs, labels = inputs.to('cuda'), labels.to('cuda')

            optimizer.zero_grad()

            # Forward and backward passes
            outputs = model.forward(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if steps % print_every == 0:
                print("Epoch: {}/{}... ".format(e+1, epochs),
                      "Loss: {:.4f}".format(running_loss/print_every))

                running_loss = 0
    
# def check_accuracy_on_test(testloader):    
#     correct = 0
#     total = 0
#     with torch.no_grad():
#         for data in testloader:
#             images, labels = data
#             outputs = model(images)
#             _, predicted = torch.max(outputs.data, 1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()

#     print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
    
    

#check_accuracy_on_test(testloader)

In [ ]:
do_deep_learning(model, trainloader, 3, 40, criterion, optimizer, 'gpu')

In [6]:
def check_accuracy_on_test(loader_): 
   #data=testloader
    
    correct = 0
    total = 0
    with torch.no_grad():
        for data in loader_:
            images, labels = data
            images, labels = images.to('cuda'), labels.to('cuda')
            #inputs.to('cuda'), labels.to('cuda')
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
    
check_accuracy_on_test(trainloader)    

Accuracy of the network on the 10000 test images: 94 %


In [7]:
def check_accuracy_on_test_(loader_): 
   #data=testloader
    
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for data in loader_:
            images, labels = data
            images, labels = images.to('cuda'), labels.to('cuda')
            #inputs.to('cuda'), labels.to('cuda')
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
    
check_accuracy_on_test_(trainloader)    

Accuracy of the network on the 10000 test images: 94 %


In [8]:
    
check_accuracy_on_test_(testloader)  

Accuracy of the network on the 10000 test images: 98 %


In [9]:
check_accuracy_on_test(testloader)  

Accuracy of the network on the 10000 test images: 98 %


In [39]:
model2 = models.resnet152(pretrained=True)
model2

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [57]:
model2 = models.resnet152(pretrained=True)

for param in model2.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(2048, 1024)),
                          ('relu1', nn.ReLU()),
                          ('fc2', nn.Linear(1024, 256)),
                          ('relu2', nn.ReLU()),    
                          ('fc3', nn.Linear(256, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model2.fc = classifier   #last layer of resnet called fc not classifier so removing model.classifier with model.fc

criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model2.fc.parameters(), lr=1e-3) #, betas=(0.9, 0.999), eps=1e-8,
 

In [58]:
model2

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [59]:
do_deep_learning(model2, trainloader, 1, 40, criterion, optimizer, 'gpu')

Epoch: 1/1...  Loss: 0.2770
Epoch: 1/1...  Loss: 0.1744
Epoch: 1/1...  Loss: 0.1424
Epoch: 1/1...  Loss: 0.1438
Epoch: 1/1...  Loss: 0.1236
Epoch: 1/1...  Loss: 0.1301
Epoch: 1/1...  Loss: 0.1397
Epoch: 1/1...  Loss: 0.1297


In [60]:
def check_accuracy_on_test__(loader_): 
   #data=testloader
    
    correct = 0
    total = 0
    model2.eval()
    with torch.no_grad():
        for data in loader_:
            images, labels = data
            images, labels = images.to('cuda'), labels.to('cuda')
            #inputs.to('cuda'), labels.to('cuda')
            outputs = model2(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
    
#check_accuracy_on_test__(trainloader)    
#check_accuracy_on_test__(testloader) 

#densenet without eval  resnet 18 has fewer input layer to classifer
#Accuracy of the network on the 10000 train images: 94 %
#Accuracy of the network on the 10000 test images: 98 %


#resnet 18 without eval  resnet 18 has fewer input layer to classifer
#Accuracy of the network on the 10000 train images: 93 %
#Accuracy of the network on the 10000 test images: 52 %  ?

#resnet 152 ? 2048 1024 256 2    ??? 
#Accuracy of the network on the 10000 train images: 95 %
#Accuracy of the network on the 10000 test images: 81 % ?


#resnet 152 ? 2048 1024 256 2    2nd round
#Accuracy of the network on the 10000 train images: 95 % 
#Accuracy of the network on the 10000 test images: 98 %

In [61]:
check_accuracy_on_test__(testloader) 

Accuracy of the network on the 10000 test images: 98 %


In [62]:
check_accuracy_on_test__(trainloader) 

Accuracy of the network on the 10000 test images: 95 %
